#GIL

К сожалению, в Python и его стандартном интерпретаторе CPython только кажется, что потоки выполняются параллельно, на самом деле они выполняются последовательно. Это связано с GIL (Global Interpreter Lock), который ограничивает Python на один запущенный поток в единицу времени.

В предыдущем модуле, посвященном сборщику мусора, было описано, что в Python существует система подсчета ссылок на объекты. Проблема, которую решает GIL, связана с возможностью одновременного увеличения или уменьшения ссылок на объекты разными потоками. Может возникнуть ситуация, когда один поток уменьшит число ссылок на объект и Python удалит его, а другой поток будет использовать только что удаленный объект, что приведет к ошибке.

В теории такая проблема может быть решена добавлением блокировок к каждому объекту, но, к сожалению, это может привести к проблеме взаимоблокировок — это когда потоки будут находится в режиме ожидания ресурсов, который захватил другой поток, и так бесконечно.

__GIL — это блокировка самого интерпретатора Python. То есть, она является единственной блокировкой в системе и позволяет решить проблему взаимоблокировок, но в свою очередь делает все приложения однопоточными.__

Если разделить все программы на CPU-зависимые (обработка изображений, умножение матриц) и I/O-зависимые (связь по сети, обращение к БД), то можно понять, что использование потоков и GIL не несет ничего критического при I/O-операциях, т.к. время, затраченное Python на переключение потоков, будет компенсировано временем I/O-операций. При этом естественно, что, в не зависимости от количества ядер процессора, любая многопоточная программа на Python не сможет раскрыть потенциал и будет работать даже медленнее однопоточной за счет переключения GIL между потоками.

Рассмотрев GIL, следует обратиться к Python-модулю __threading__, который отвечает за создание и работу с потоками.

In [ ]:
import threading
import time

def thread_function(name):
  print(name, ": thread starting")
  time.sleep(2)
  print(name, ": after sleep")


if __name__ == "__main__":
  print("Before create Thread")
  x = threading.Thread(target=thread_function, args=(1,))
  print("Before running Thread")
  x.start()
  print("Wait Thread finish")
  x.join()
  print("All done")

Before create Thread
Before running Thread
1 : thread starting
Wait Thread finish
1 : after sleep
All done


В данном примере:

1. Мы импортируем модуль __threading__.
2. Создаем объект класса __Thread__, передавая ему на вход функцию, с которой он начнет работу, и аргументы для этой функции. 
3. Методом __start()__ можно запустить поток, и, когда он завершит выполнение функции __thread_function()__, он автоматически завершится.

Также во время создания объекта класса __Thread__ можно передать параметр __daemon=True__, который позволит создать daemon-поток.

In [ ]:
x = threading.Thread(target=thread_function, args=(1,), daemon=True)


В теории daemon-процесс — это процесс, который работает в фоновом режиме.

В Python различают обычные потоки и daemons-потоки. Приложение для остановки будет ждать корректного завершения обычных потоков, daemons-потоки же будут просто убиты. Можно представить, что daemon-поток — это фоновый поток, о завершении которого можно не беспокоиться.

__x.join()__ — это указание основному потоку дождаться завершения потока x. Это может быть полезно в случае, когда дочерние потоки делают какую-то работу, а основной поток впоследствии работает с данными, которые подготовили дочерние потоки.

Пример, который описан выше, позволяет создавать один поток, а для запуска нескольких можно комбинировать их, помещая в список:

In [ ]:
import threading
import time

def thread_function(name):
  print(name, ": thread starting\n")
  time.sleep(2)
  print(name, ": job is done\n")

if __name__ == "__main__":
  threads = []
  for i in range(3):
    print("create thread - ", i, "\n")
    x = threading.Thread(target=thread_function, args=(i,))
    threads.append(x)
    x.start()

for i, thread in enumerate(threads):
  print("before join - ", i, "\n")
  thread.join()
  print("after join - ", i, "\n")

create thread -  0 

0 : thread starting

create thread -  1 

1 : thread starting

create thread -  2 

2before join -  : thread starting

 0 

0 : job is done

after join -  0 

before join -  1 

1 : job is done

after join -  21 

before join -  2 

 : job is done

after join -  2 



Можно видеть, что, после старта потоков, главный поток зависает в ожидании потока под номером 0. Но потоки почти в одинаковое время завершают работу, поэтому остальные join() проходят быстро.

Помимо создания нескольких потоков и хранения их в списке, в Python есть возможность использовать __ThreadPoolExecutor__, который позволяет создать N потоков более просто.

In [2]:
from concurrent.futures import ThreadPoolExecutor
import threading
import time

def thread_function():
  time.sleep(2)
  print("thread_function Executed {}".format(threading.current_thread()))

def main():
  executor = ThreadPoolExecutor(max_workers=3)
  task1 = executor.submit(thread_function)
  task2 = executor.submit(thread_function)

if __name__ == '__main__':
  main()

In [ ]:
thread_function Executed <Thread(ThreadPoolExecutor-0_0, started daemon 123145431654400)>
thread_function Executed <Thread(ThreadPoolExecutor-0_1, started daemon 123145448443904)>

В этом примере создается ThreadPoolExecutor, с количеством потоков равным 3, и с помощью объекта executor передается функция, которую нужно выполнить. Как видно из вывода, разные потоки выполняют эту функцию.